In [4]:
%%capture
!pip install -U sentence-transformers
!pip install pinecone-client


In [5]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from ast import literal_eval

cargamos las librerias a utilizar

In [6]:
peliculas= pd.read_csv('pelis.csv')
peliculas.head(3)

,movie title,Run Time,Rating,User Rating,Generes,Overview,Plot Kyeword,Director,Top 5 Casts,Writer,year,path
0,Top Gun: Maverick,"$170,000,000 (estimated)",8.6,187K,"['Action', 'Drama']",After more than thirty years of service as one...,"['fighter jet', 'sequel', 'u.s. navy', 'fighte...",Joseph Kosinski,"['Jack Epps Jr.', 'Peter Craig', 'Tom Cruise',...",Jim Cash,-2022,/title/tt1745960/
1,Jurassic World Dominion,2 hours 27 minutes,6,56K,"['Action', 'Adventure', 'Sci-Fi']",Four years after the destruction of Isla Nubla...,"['dinosaur', 'jurassic park', 'tyrannosaurus r...",Colin Trevorrow,"['Colin Trevorrow', 'Derek Connolly', 'Chris P...",Emily Carmichael,-2022,/title/tt8041270/
2,Top Gun,"$15,000,000 (estimated)",6.9,380K,"['Action', 'Drama']",As students at the United States Navy's elite ...,"['pilot', 'male camaraderie', 'u.s. navy', 'gr...",Tony Scott,"['Jack Epps Jr.', 'Ehud Yonay', 'Tom Cruise', ...",Jim Cash,-1986,/title/tt0092099/


podemos ver que en su mayoria los datos son texto como por ejemplo las keywords hacemos una funcion para pasarlas a una lista

In [7]:
def concatenar_lista(lista):
    # Convertimos la cadena de entrada en una lista de Python utilizando literal_eval.
    lista = literal_eval(lista)

    # Usamos el método join para concatenar los elementos de la lista en una cadena de texto.
    # Se utiliza un espacio como separador entre los elementos.
    return ' '.join(lista)

In [8]:
def string_to_list(lista):

    lista = literal_eval(lista)
    return lista

ahora aplicamos las funciones a cada una de las columnas

In [9]:
peliculas = peliculas.fillna(' ') # le decimos que los espacios nulos no los coloco con un espacio vacio
peliculas['Keywords'] = peliculas['Plot Kyeword'].apply(concatenar_lista) #creamos una nueva columna apartir de plot keywords pero en formato de str
peliculas['Stars'] = peliculas['Top 5 Casts'].apply(concatenar_lista)#creamos una nueva columna apartir de top 5  pero en formato de str
peliculas['Generes'] = peliculas['Generes'].apply(string_to_list) #creamos una nueva columna apartir de generes pero en formato de lista
peliculas['Rating'] = pd.to_numeric(peliculas['Rating'], errors="coerce").fillna(0).astype("float") # el rating esta en texto lo pasamos a numeros

In [10]:
peliculas.head(3)

,movie title,Run Time,Rating,User Rating,Generes,Overview,Plot Kyeword,Director,Top 5 Casts,Writer,year,path,Keywords,Stars
0,Top Gun: Maverick,"$170,000,000 (estimated)",8.6,187K,"[Action, Drama]",After more than thirty years of service as one...,"['fighter jet', 'sequel', 'u.s. navy', 'fighte...",Joseph Kosinski,"['Jack Epps Jr.', 'Peter Craig', 'Tom Cruise',...",Jim Cash,-2022,/title/tt1745960/,fighter jet sequel u.s. navy fighter aircraft ...,Jack Epps Jr. Peter Craig Tom Cruise Jennifer ...
1,Jurassic World Dominion,2 hours 27 minutes,6.0,56K,"[Action, Adventure, Sci-Fi]",Four years after the destruction of Isla Nubla...,"['dinosaur', 'jurassic park', 'tyrannosaurus r...",Colin Trevorrow,"['Colin Trevorrow', 'Derek Connolly', 'Chris P...",Emily Carmichael,-2022,/title/tt8041270/,dinosaur jurassic park tyrannosaurus rex veloc...,Colin Trevorrow Derek Connolly Chris Pratt Bry...
2,Top Gun,"$15,000,000 (estimated)",6.9,380K,"[Action, Drama]",As students at the United States Navy's elite ...,"['pilot', 'male camaraderie', 'u.s. navy', 'gr...",Tony Scott,"['Jack Epps Jr.', 'Ehud Yonay', 'Tom Cruise', ...",Jim Cash,-1986,/title/tt0092099/,pilot male camaraderie u.s. navy grumman f 14 ...,Jack Epps Jr. Ehud Yonay Tom Cruise Tim Robbin...


ahora sacamos los generos unicos

In [11]:
unique_generes = peliculas['Generes'].explode().unique()
unique_generes

array(['Action', 'Drama', 'Adventure', 'Sci-Fi', 'Animation', 'Crime',
       'Comedy', 'Thriller', 'Fantasy', 'Horror', 'History', 'Mystery',
       'Biography', 'War', 'Western', 'Sport', 'Family', 'Romance',
       'Music', 'Musical', 'Film-Noir', 'Game-Show', 'Adult',
       'Reality-TV'], dtype=object)

borramos las columnas que no vamos a necesitar

In [12]:
peliculas.drop(['Plot Kyeword','Top 5 Casts'],axis=1, inplace=True)

ahora creamos una columna que va a tener la concatenacion de el 'overview' que es la descripcion + las 'keywords' que son las palabras claves y las estrellas y esta columna es la que llevaremos a un espacio vectorial

In [13]:
peliculas['text'] = peliculas.apply(lambda x : str(x['Overview'])+' '+x['Keywords']+' '+x['Stars'], axis=1)

In [14]:
peliculas.head(2)

,movie title,Run Time,Rating,User Rating,Generes,Overview,Director,Writer,year,path,Keywords,Stars,text
0,Top Gun: Maverick,"$170,000,000 (estimated)",8.6,187K,"[Action, Drama]",After more than thirty years of service as one...,Joseph Kosinski,Jim Cash,-2022,/title/tt1745960/,fighter jet sequel u.s. navy fighter aircraft ...,Jack Epps Jr. Peter Craig Tom Cruise Jennifer ...,After more than thirty years of service as one...
1,Jurassic World Dominion,2 hours 27 minutes,6.0,56K,"[Action, Adventure, Sci-Fi]",Four years after the destruction of Isla Nubla...,Colin Trevorrow,Emily Carmichael,-2022,/title/tt8041270/,dinosaur jurassic park tyrannosaurus rex veloc...,Colin Trevorrow Derek Connolly Chris Pratt Bry...,Four years after the destruction of Isla Nubla...


descargamos el modelo que vamos a utilizar de la libreria transformer

In [15]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

hacemos el encode

In [16]:
embeddings = model.encode(peliculas['text'],batch_size=64,show_progress_bar=True)

Batches:   0%|          | 0/382 [00:00<?, ?it/s]

In [17]:
peliculas['embeddings'] = embeddings.tolist() # llevamos nuestros embeddings a el dataset en forma de lista
peliculas['ids'] = peliculas.index # creamos un indice para cada columna
peliculas['ids'] = peliculas['ids'].astype('str') # transformamos el indice en texto

ahora lo ingresamos a nuestra base de datos vectorial

In [18]:
import pinecone
from getpass import getpass

In [19]:
pincone_api = getpass('Enter the secret value: ')

Enter the secret value: ··········


conectamos a nuestro entorno de base de datos

In [20]:
pinecone.init(api_key=pincone_api, environment="gcp-starter")

verificamos la longitud de nuestros vectores de embeddings para subirlos

In [21]:
dimensions_embeddings = len(peliculas['embeddings'][0])
dimensions_embeddings

384

y subimos a nuestra base de datos todos los embeddings

In [22]:
'''creamoos una pequeña funcion para crear el indice en pinecone , y si ya esta creado solo queremos
que lo traiga '''

index_name = 'movies-embeddings' #nombre del indice
all_index = pinecone.list_indexes() # nombre de los indices ya existentes
if index_name in all_index:
    index = pinecone.Index(index_name)# si el indice existe entonces esta sera el indice
else:
    pinecone.create_index(index_name, dimension=dimensions_embeddings, metric="cosine")#sino existe crealo
    #y traelo como indice, pasamos el nombre, las dimensiones , y la metrica de distancia que tendra
    index = pinecone.Index(index_name)

In [23]:
from tqdm.auto import tqdm

# we will use batches of 64
batch_size = 64

for i in tqdm(range(0, len(peliculas), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(peliculas))
    # extract batch
    batch = peliculas[i:i_end]
    # generate embeddings for batch
    ids = batch['ids']
    emb = batch['embeddings']
    metadata = batch.drop(['ids','embeddings','text','path'],axis=1).to_dict('records')

    # add all to upsert list
    to_upsert = list(zip(ids, emb,metadata))
    # upsert/insert these records to pinecone
    _ = index.upsert(vectors=to_upsert)

# check that we have all vectors in index
index.describe_index_stats()

  0%|          | 0/382 [00:00<?, ?it/s]

{'dimension': 384,
 'index_fullness': 0.24402,
 'namespaces': {'': {'vector_count': 24402}},
 'total_vector_count': 24402}

creamos una funcion que nos permita hacer una consulta directa a nuestra base de datos

In [24]:
def search(query, genre, rating, top_k):
    # 1. Encodeamos la consulta de búsqueda en un vector utilizando un modelo previamente entrenado (no se muestra la definición de 'model' en el código).

    query_vector = model.encode(query).tolist()

    # 2. Configuramos las condiciones de filtrado basadas en el género y la calificación.
    if rating:
        filter_rating = rating #si nos dan un rating lo seleccionamos
    else:
        filter_rating = 0# sino que el rating sea 0

    if genre:
        conditions = {
            "Generes": { "$in": [genre] },
            "Rating": { "$gte": filter_rating }
        }
    else:
        conditions = {
            "Rating": { "$gte": filter_rating }
        }

    # 3. Realizamos una consulta en la base de datos vectorial de Pinecone utilizando el vector de consulta y las condiciones de filtrado.
    responses = index.query(
        vector=query_vector,
        top_k=top_k,
        include_metadata=True,
        filter=conditions
    )

    # 4. Formateamos las respuestas para su visualización.
    response_data = []
    for response in responses['matches']:
        response_data.append({
            'Title': response['metadata']['movie title'],
            'Overview': response['metadata']['Overview'],
            'Director': response['metadata']['Director'],
            'Genre': response['metadata']['Generes'],
            'Year': response['metadata']['year'],
            'Rating': response['metadata']['Rating'],
            'Score': response['score'],
        })

    # 5. Creamos un DataFrame de pandas a partir de los datos formateados y lo devuelve como resultado.
    df = pd.DataFrame(response_data)
    return df

hacemos una consulta para buscar un tipo de pelicula

In [28]:
search('a time to horror',None, None,5)

,Title,Overview,Director,Genre,year,Rating,Score
0,Dead & Buried,A suspense horror film set in a small coastal ...,Gary Sherman,"[Horror, Mystery]",-1981,6.5,0.553277
1,The Haunting,A study in fear escalates into a heart-stoppin...,Jan de Bont,"[Fantasy, Horror, Mystery]",-1999,5.0,0.509689
2,Two Eyes Staring,A horror tale centered on the friendship betwe...,Scott Derrickson,"[Horror, Mystery, Thriller]",,0.0,0.507578
3,The Strangers: Prey at Night,A family of four staying at a secluded mobile ...,Johannes Roberts,[Horror],-2018,5.2,0.504407
4,Midnight Movie,A midnight showing of an early 1970's horror m...,Jack Messitt,"[Fantasy, Horror, Thriller]",-2008,5.0,0.493084
